## Convert Latitudes and Longitudes to Web Mercator Co-ordinates

Folium visualization was not working properly in my conda installation. Hence as an alternative I am using Bokeh Geospatial Mapping to show the results and visualizations. Bokeh uses the web mercator projection for display and a different format than GeoDataframe. Hence this file handles the transformations and exports the data as JSON for easy access.

In [123]:
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
import json as JSON
import pandas as pd
import shapely
from IPython.display import clear_output
output_notebook() 

Loading BokehJS ...

In [3]:
def convert_GeoPandas_to_Bokeh_format(gdf):
    gdf_new = gdf.drop('geometry', axis=1).copy()
    gdf_new['x'] = gdf.apply(getGeometryCoords, 
                             geom='geometry', 
                             coord_type='x', 
                             shape_type='polygon', 
                             axis=1)
    
    gdf_new['y'] = gdf.apply(getGeometryCoords, 
                             geom='geometry', 
                             coord_type='y', 
                             shape_type='polygon', 
                             axis=1)
    return gdf_new
    return ColumnDataSource(gdf_new)


def getGeometryCoords(row, geom, coord_type, shape_type):
    # Parse the exterior of the coordinate
    if shape_type == 'polygon':
        if(type(row[geom])==shapely.geometry.polygon.Polygon):
            exterior = row[geom].exterior
        elif(type(row[geom])==shapely.geometry.multipolygon.MultiPolygon):
            exterior = row[geom].geoms[1].exterior
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            return list( exterior.coords.xy[0] )    
        
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( exterior.coords.xy[1] )

    elif shape_type == 'point':
        exterior = row[geom]
    
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            return  exterior.coords.xy[0][0] 

        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return  exterior.coords.xy[1][0]



In [16]:
newdf=pd.read_json("Data.json",orient="table")
newdf.head()

,code,name,lat,long,area,borough,population,medAge,popDensity,dayPop,...,Japanese Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Nightclub/Lounge,Pizza Place,Seafood Restaurant,Steakhouse,culturalFactor,x,y
0,BK09,Brooklyn Heights-Cobble Hill,40.695469,-73.994871,1.615327,Brooklyn,24212,37.1,14988,21307.0,...,6.0,4.0,2.0,2.0,6.0,0.0,1.0,24.125417,"[-73.9923636704, -73.9943609283, -73.996232780...","[40.6896901238, 40.6902394801, 40.6907577746, ..."
1,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,40.588300,-73.941511,10.214922,Brooklyn,67681,43.9,6625,59559.0,...,9.0,1.0,8.0,3.0,10.0,6.0,0.0,16.608750,"[-73.9180925648, -73.9181253337, -73.918055685...","[40.586570335, 40.5865073745, 40.5864365457, 4..."
2,BK19,Brighton Beach,40.580922,-73.961217,2.770061,Brooklyn,35811,44.3,12927,31514.0,...,4.0,1.0,2.0,2.0,2.0,1.0,0.0,3.000000,"[-73.9603495359, -73.9603171203, -73.960317663...","[40.5873062856, 40.5865147236, 40.586436468, 4..."
3,BK21,Seagate-Coney Island,40.576480,-73.991231,6.242851,Brooklyn,31132,39.0,4986,27396.0,...,0.0,10.0,1.0,3.0,10.0,2.0,0.0,38.950583,"[-73.9745900058, -73.9745935098, -73.974586671...","[40.5831388208, 40.5828996033, 40.5828446826, ..."
4,BK23,West Brighton,40.579088,-73.973391,1.409979,Brooklyn,16436,58.0,11656,14464.0,...,1.0,0.0,1.0,1.0,4.0,0.0,0.0,0.000000,"[-73.9688899588, -73.9689458946, -73.969037798...","[40.575261239, 40.5752786881, 40.5753086191, 4..."


### Convert Lats and Longs to Mercator

In [39]:
from pyproj import Proj, transform
from functools import partial
def xytoMerc(x,y):
    merc=partial(transform,Proj(init='epsg:4326'), Proj(init='epsg:3857'))
    return merc(x,y)
cord=newdf.loc[0,['x','y']]

In [145]:
def getPatch(row):
    mercs=[]
    for i in zip(*(row['x'],row['y'])):
        mercs.append(xytoMerc(i[0],i[1]))
    a,b=zip(*mercs)
    return (list(a),list(b))

xs=[]
ys=[]
for i,row in newdf.iterrows():
    x,y=getPatch(row)
    xs.append(x)
    ys.append(y)
    clear_output()
    print("Rows completed:",i)

Rows completed: 187


### Export processed Data

In [158]:
merDf=newdf.copy()
merDf['merc_x']=xs
merDf['merc_y']=ys
data=ColumnDataSource(merDf)
# data=dict(x=xs,y=ys)
merDf.to_json("mapping.json",orient="table")